<a href="https://colab.research.google.com/github/jamelof23/Finance/blob/main/Final%26LAST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**FMP**

*   change program to less than 750 per minList item
*   change program to less than 750 per min
*   speed is 750 per min , i did 16k in 8 min
* changed to 700 per min, Specs is 750


In [ ]:
import requests
import pandas as pd
import concurrent.futures
import time
import openpyxl

# Your Financial Modeling Prep API key
API_KEY = 'KcsOu8cJesk2GpAJ0DXN4a5Il3DlLUX1'
BASE_URL = 'https://financialmodelingprep.com/api/v4/insider-trading'
ALL_SYMBOLS_URL = f"https://financialmodelingprep.com/api/v3/stock/list?apikey={API_KEY}"

RATE_LIMIT = 700  # Requests per minute
DELAY_BETWEEN_REQUESTS = 60 / RATE_LIMIT  # Delay in seconds to respect the rate limit

def fetch_all_symbols():
    """Fetch all available U.S. stock symbols from NASDAQ and NYSE."""
    try:
        response = requests.get(ALL_SYMBOLS_URL)
        response.raise_for_status()
        data = response.json()
        symbols = [item['symbol'] for item in data if item['exchangeShortName'] in ['NASDAQ', 'NYSE']]
        return symbols
    except Exception as e:
        print(f"Error fetching all symbols: {e}")
        return []

def fetch_insider_trading(symbol):
    """Fetch insider trading data for a given symbol with retry logic."""
    url = f"{BASE_URL}?symbol={symbol}&apikey={API_KEY}"
    retries = 3  # Retry up to 3 times
    for attempt in range(retries):
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            if data:
                return symbol, data
            else:
                return symbol, None
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(1)  # Wait 1 second before retrying
            else:
                print(f"Failed to fetch data for {symbol} after {retries} attempts.")
                return symbol, None

def fetch_all_data(symbols):
    """Fetch insider trading data for all symbols using parallel processing."""
    fetched_data = []
    total_symbols = len(symbols)
    not_fetched_symbols = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(fetch_insider_trading, symbol): symbol for symbol in symbols}
        completed = 0

        for future in concurrent.futures.as_completed(futures):
            symbol, data = future.result()
            if data:
                for transaction in data:
                    transaction['Symbol'] = symbol
                    fetched_data.append(transaction)
                print(f"Fetched data for {symbol} successfully.")
            else:
                not_fetched_symbols.append(symbol)
                print(f"No data found for {symbol}.")

            completed += 1
            print(f"{completed} out of {total_symbols} symbols completed.")

            # Respect rate limit
            time.sleep(DELAY_BETWEEN_REQUESTS)

    return fetched_data, not_fetched_symbols

def save_to_excel(data, filename='Insider_Transactions_FMP.xlsx'):
    """Save fetched data to an Excel file."""
    df = pd.DataFrame(data)
    df.to_excel(filename, index=False)
    print(f"\nData saved to {filename}")

def save_symbols_to_excel(symbols, not_fetched, filename='All_Symbols_FMP.xlsx'):
    """Save all symbols to an Excel file, with an additional column for data fetched status."""
    df = pd.DataFrame({'Symbol': symbols, 'Data_Fetched': [symbol not in not_fetched for symbol in symbols]})
    df.to_excel(filename, index=False)
    print(f"\nSymbols saved to {filename}")

# Step 1: Fetch all U.S. stock symbols
symbols = fetch_all_symbols()
print(f"Total symbols fetched: {len(symbols)}")

# Step 2: Fetch insider trading data for all symbols and display progress
start_time = time.time()
fetched_data, not_fetched_symbols = fetch_all_data(symbols)
end_time = time.time()

print(f"\nFetched insider trading data for {len(fetched_data)} entries in {end_time - start_time:.2f} seconds.")

# Step 3: Save the fetched data to an Excel file
save_to_excel(fetched_data, 'Insider_Transactions_FMP.xlsx')

# Step 4: Save all symbols to a separate Excel file, indicating whether data was fetched or not
save_symbols_to_excel(symbols, not_fetched_symbols, 'All_Symbols_FMP.xlsx')



**Free Code from Fin hub**

* today 6 nov , last data was 15 oct available from 1 nov in yahoo
* need progress for what is completed how much remaining
* need auto download file
* G gift for elon why minus should be plus added stovk no value eventhough he filed 3 months later
* S sell G gift and minus what is Buy (try put stock has bought if B appears)

* need to upload file for all symbols
* speed is 55 per min, specs is 60
* Free only 50 Symbols




In [ ]:
# Install required libraries
!pip install finnhub-python openpyxl

import finnhub
import pandas as pd
import time
from google.colab import files
import requests

# Replace 'YOUR_API_KEY' with your actual Finnhub API key
API_KEY = 'csle2g1r01qq49fgr7lgcsle2g1r01qq49fgr7m0'

# Initialize Finnhub client
finnhub_client = finnhub.Client(api_key=API_KEY)

# Load symbols from your Excel file using read_excel
symbols_df = pd.read_excel('/content/All_Symbols_FMP.xlsx')
symbol_list = symbols_df['Symbol'].tolist()  # Assuming the column name is 'Symbol'


# Function to fetch insider transactions with rate limiting (55 requests per minute)
def fetch_insider_transactions(symbols, rate_limit_per_minute=55):
    insider_data = []
    start_time = time.time()
    requests_made = 0
    delay_between_requests = 60 / rate_limit_per_minute  # Calculate delay to maintain 55 requests per minute

    for idx, symbol in enumerate(symbols):
        try:
            # Fetch insider transactions with '_from' date
            data = finnhub_client.stock_insider_transactions(symbol, _from='2020-01-01')
            # Check if data is available
            if 'data' in data and data['data']:
                for transaction in data['data']:
                    insider_data.append(transaction)
            else:
                print(f"No insider transactions for symbol {symbol}")
        except requests.exceptions.HTTPError as http_err:
            if http_err.response.status_code == 429:
                print("Rate limit exceeded. Sleeping for a minute.")
                time.sleep(60)
            else:
                print(f"HTTP error occurred for symbol {symbol}: {http_err}")
        except Exception as e:
            print(f"Error fetching data for symbol {symbol}: {e}")

        requests_made += 1

        # Implement rate limiting
        if requests_made >= rate_limit_per_minute:
            elapsed_time = time.time() - start_time
            if elapsed_time < 60:
                time.sleep(60 - elapsed_time)
            start_time = time.time()
            requests_made = 0
        else:
            time.sleep(delay_between_requests)  # Sleep to maintain the rate of 55 requests per minute

        # Progress update
        if (idx + 1) % 100 == 0:
            print(f"Processed {idx + 1}/{len(symbols)} symbols")
    return insider_data

# Initialize insider_transactions
insider_transactions = []

try:
    # Fetch insider transactions
    insider_transactions = fetch_insider_transactions(symbol_list)

except KeyboardInterrupt:
    print("\nProcess interrupted by user. Saving data collected so far...")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Saving data collected so far...")

finally:
    if insider_transactions:
        # Convert to DataFrame
        df = pd.DataFrame(insider_transactions)

        # Save to Excel file
        excel_file_name = 'insider_transactions.xlsx'
        df.to_excel(excel_file_name, index=False)

        # Download the Excel file to your local computer
        files.download(excel_file_name)
        print(f"Data saved to {excel_file_name} and downloaded to your local computer.")
    else:
        print("No data collected to save.")
